In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
data_dir = '/Users/shivyucel/Documents/projects.nosync/opioid_accessibility/opioid_accessibility/data/toronto_data/'

In [2]:
LICOdata = pd.read_csv(data_dir + 'LICODATADALevel.csv')

In [3]:
toronto_das = LICOdata[LICOdata['COL4'] == 'Toronto']

In [4]:
carless = gpd.read_file('/Users/shivyucel/Documents/projects.nosync/opioid_accessibility/opioid_accessibility/data/toronto_data/GTAWithVarsForMaps/GTAWithVarsForMaps.shp')
employ = pd.read_csv(data_dir + 'DA employment counts/DA_EMP_2016.csv')
travel_diff = gpd.read_file(data_dir + 'results/travel_time_diff/travel_time_diff_2.shp')
clinics_gdf = gpd.read_file('/Users/shivyucel/Documents/projects.nosync/opioid_accessibility/opioid_accessibility/data/toronto_data/clinic_shp/clean_clinics_v3.shp')
low_income_cutoff = pd.read_csv('/Users/shivyucel/Documents/projects.nosync/opioid_accessibility/opioid_accessibility/data/toronto_data/low_income_cutoff/LICODataDALevel.csv')
DA_income = pd.read_csv(data_dir + 'census_data/DA_income.csv')

In [5]:
carless['DAUID'] = carless['DAUID'].astype(int)

In [6]:
carless = carless[carless['DAUID'].isin(toronto_das['COL0'])]

In [7]:
carless = carless.merge(DA_income, left_on='DAUID', right_on='GeoUID')

In [8]:
carless['PCTCarless'] = carless['PCTCarless'] * 0.01

In [9]:
carless['pop_carless'] = carless['PCTCarless'] * carless['Population']

In [10]:
carless

,DAUID,PRUID,PRNAME,CDUID,CDNAME,CDTYPE,CCSUID,CCSNAME,CSDUID,CSDNAME,...,Population,Dwellings,Households,CSD_UID,CD_UID,CT_UID,CMA_UID,v_CA16_4986: Average after-tax income of households in 2015 ($),"v_CA16_401: Population, 2016",pop_carless
0,35201588,35,Ontario,3520,Toronto,CDR,3520005,Toronto,3520005,Toronto,...,608,234,232,3520005,3520,5350211.00,35535,76468.0,608,153.405524
1,35201589,35,Ontario,3520,Toronto,CDR,3520005,Toronto,3520005,Toronto,...,689,252,251,3520005,3520,5350211.00,35535,99229.0,689,0.000000
2,35201590,35,Ontario,3520,Toronto,CDR,3520005,Toronto,3520005,Toronto,...,571,241,224,3520005,3520,5350211.00,35535,75193.0,571,21.502379
3,35201591,35,Ontario,3520,Toronto,CDR,3520005,Toronto,3520005,Toronto,...,620,245,229,3520005,3520,5350212.00,35535,68993.0,620,183.608969
4,35201592,35,Ontario,3520,Toronto,CDR,3520005,Toronto,3520005,Toronto,...,410,179,165,3520005,3520,5350211.00,35535,85357.0,410,74.995551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3697,35204746,35,Ontario,3520,Toronto,CDR,3520005,Toronto,3520005,Toronto,...,1383,372,368,3520005,3520,5350378.11,35535,73154.0,1383,441.921532
3698,35204747,35,Ontario,3520,Toronto,CDR,3520005,Toronto,3520005,Toronto,...,1544,481,471,3520005,3520,5350378.11,35535,69247.0,1544,39.399697
3699,35204748,35,Ontario,3520,Toronto,CDR,3520005,Toronto,3520005,Toronto,...,899,516,509,3520005,3520,5350010.01,35535,124492.0,899,251.828893
3700,35204749,35,Ontario,3520,Toronto,CDR,3520005,Toronto,3520005,Toronto,...,1088,755,733,3520005,3520,5350010.01,35535,78306.0,1088,350.858181


In [11]:
carless = carless.merge(travel_diff, left_on='DAUID', right_on='fromId', how='left')

In [12]:
carless.drop_duplicates('DAUID')['pop_carless'].sum()

658591.2636332931

In [13]:
clean_times = carless.pivot(index='DAUID', columns='time', values='diff')

In [14]:
carless.drop_duplicates('DAUID', inplace=True)

In [15]:
clean_times = clean_times.merge(carless[['DAUID', 'Region Name', 'Area (sq km)', 'Population', 'v_CA16_4986: Average after-tax income of households in 2015 ($)', 'pop_carless']] , left_index=True, right_on='DAUID')

In [16]:
clean_times

,NaN,0800,0900_sat,1100,1500,DAUID,Region Name,Area (sq km),Population,v_CA16_4986: Average after-tax income of households in 2015 ($),pop_carless
7550,NaN,0.0,0.0,-3.0,-1.0,35200002,Toronto,0.05037,356,79431.0,55.071330
8386,NaN,1.0,0.0,-3.0,-1.0,35200003,Toronto,0.04675,335,97695.0,0.000000
8390,NaN,0.0,0.0,-3.0,-2.0,35200004,Toronto,0.04878,384,66349.0,0.000000
8550,NaN,-1.0,0.0,-3.0,-1.0,35200005,Toronto,0.04429,408,97356.0,0.000000
8554,NaN,-1.0,0.0,-3.0,-1.0,35200006,Toronto,0.05717,527,80188.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
14515,NaN,0.0,0.0,0.0,0.0,35204902,Toronto,0.01994,498,55037.0,321.265496
14735,NaN,2.0,3.0,2.0,3.0,35204903,Toronto,0.63493,3495,48495.0,494.285900
14739,NaN,2.0,2.0,1.0,2.0,35204904,Toronto,0.08969,626,51243.0,292.351880
14743,NaN,1.0,1.0,1.0,1.0,35204905,Toronto,1.08844,1774,50783.0,307.684076


In [17]:
low_income_cutoff = pd.read_csv('/Users/shivyucel/Documents/projects.nosync/opioid_accessibility/opioid_accessibility/data/toronto_data/low_income_cutoff/LICODataDALevel.csv')

In [18]:
clean_times = clean_times.merge(low_income_cutoff[['COL0', 'COL7']], left_on='DAUID', right_on='COL0')

In [19]:
clean_times = clean_times.rename(columns={'0800': 'Wednesday 0800', 
                                          '0900_sat': 'Saturday 0900', 
                                          '1100': 'Wednesday 1100',
                                          '1500': 'Wedensday 1500', 
                                          'v_CA16_4986: Average after-tax income of households in 2015 ($)': 'avg_after_tax_income', 
                                          'COL7': 'low_income_cutoff_pop'})

In [20]:
clean_times.drop('COL0', axis=1, inplace=True)

In [21]:
carless['geometry_x']

0        POLYGON ((7212896.883 920397.774, 7212811.303 ...
4        POLYGON ((7212775.083 920625.880, 7212687.303 ...
8        POLYGON ((7213168.374 921135.129, 7213291.720 ...
12       POLYGON ((7213104.223 919396.989, 7213081.400 ...
16       POLYGON ((7213596.074 920773.111, 7213503.614 ...
                               ...                        
14779    POLYGON ((7234235.494 948139.231, 7234224.549 ...
14783    POLYGON ((7234502.834 947586.174, 7234494.777 ...
14787    POLYGON ((7222525.997 926886.283, 7222500.320 ...
14791    POLYGON ((7222719.777 926815.820, 7222716.751 ...
14795    POLYGON ((7229440.837 930428.526, 7229121.703 ...
Name: geometry_x, Length: 3702, dtype: geometry

In [22]:
clean_times = clean_times.merge(carless[['DAUID', 'geometry_x']], on='DAUID')
clean_times.rename(columns={'geometry_x': 'geometry'}, inplace=True)

In [23]:
clean_times = gpd.GeoDataFrame(clean_times, geometry='geometry')
clean_times.set_crs(epsg='3347', inplace=True)


,NaN,Wednesday 0800,Saturday 0900,Wednesday 1100,Wedensday 1500,DAUID,Region Name,Area (sq km),Population,avg_after_tax_income,pop_carless,low_income_cutoff_pop,geometry
0,NaN,0.0,0.0,-3.0,-1.0,35200002,Toronto,0.05037,356,79431.0,55.071330,25.0,"POLYGON ((7235605.171 950635.097, 7235659.626 ..."
1,NaN,1.0,0.0,-3.0,-1.0,35200003,Toronto,0.04675,335,97695.0,0.000000,35.0,"POLYGON ((7235794.929 950295.603, 7235846.557 ..."
2,NaN,0.0,0.0,-3.0,-2.0,35200004,Toronto,0.04878,384,66349.0,0.000000,40.0,"POLYGON ((7235509.514 950062.337, 7235507.757 ..."
3,NaN,-1.0,0.0,-3.0,-1.0,35200005,Toronto,0.04429,408,97356.0,0.000000,25.0,"POLYGON ((7235628.734 949921.829, 7235606.546 ..."
4,NaN,-1.0,0.0,-3.0,-1.0,35200006,Toronto,0.05717,527,80188.0,0.000000,40.0,"POLYGON ((7235705.871 949866.543, 7235688.483 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3697,NaN,0.0,0.0,0.0,0.0,35204902,Toronto,0.01994,498,55037.0,321.265496,165.0,"POLYGON ((7224597.117 929588.000, 7224434.543 ..."
3698,NaN,2.0,3.0,2.0,3.0,35204903,Toronto,0.63493,3495,48495.0,494.285900,1170.0,"POLYGON ((7222961.389 941366.423, 7222950.120 ..."
3699,NaN,2.0,2.0,1.0,2.0,35204904,Toronto,0.08969,626,51243.0,292.351880,175.0,"POLYGON ((7222913.800 941347.529, 7222901.400 ..."
3700,NaN,1.0,1.0,1.0,1.0,35204905,Toronto,1.08844,1774,50783.0,307.684076,525.0,"POLYGON ((7225992.023 942215.857, 7225991.686 ..."


In [24]:
clean_times.columns

Index([                    nan,        'Wednesday 0800',
               'Saturday 0900',        'Wednesday 1100',
              'Wedensday 1500',                 'DAUID',
                 'Region Name',          'Area (sq km)',
                  'Population',  'avg_after_tax_income',
                 'pop_carless', 'low_income_cutoff_pop',
                    'geometry'],
      dtype='object')

In [25]:
clean_times = clean_times.iloc[:, 1:]

In [ ]:
#Note: clean times has three rows without travel time difference data (due to trip being too long)

In [27]:
clean_times.columns=['wed_0800', 'sat_0900', 'wed_1100', 'wed_1500', 'DAUID', 'reg_name', 'area_km', 'pop', 'avg_inc', 'pop_nocar', 'low_inc', 'geometry']

In [29]:
clean_times.to_file('/Users/shivyucel/Documents/projects.nosync/opioid_accessibility/opioid_accessibility/data/toronto_data/clean_files/clean_times/clean_times.shp')

In [30]:
x = gpd.read_file('/Users/shivyucel/Documents/projects.nosync/opioid_accessibility/opioid_accessibility/data/toronto_data/clean_files/clean_times/clean_times.shp')